### 概要

#### 数据简介
1. `triple_zh.txt` 中文关系的三元组，共 104941 个
2. `triple_en.txt` 英文关系的三元组，共 162544 个
3. `ILLs(zh-en).txt` 中英关键词转换，共 13636 个
4. `train_data.txt` 训练集，共 14262 个，每个解答有 2-3 个三元组
5. `valid_data.txt` 验证集

#### 预处理内容
1. 提取三元组的关键词
2. 训练集分解为：问题 + 解答
3. 提取问题中的混合输入（需确定 unicode 范围，待进行）
4. 统计连接词的词频（可选，未进行）

### 关键词提取
存储在 `extract` 目录下

In [1]:
;cd ../data

/home/rex/work_space/7 others/ccks/CCKS-mKGQA/data


#### 提取文件
ILLs(zh-en).txt, triple_en.txt, triple_zh.txt

In [2]:
txt2triple(txt) = Tuple(rstrip(last(split(st, '/')), '>') for st in split(txt))

In [3]:
filename = "ILLs(zh-en).txt"
output_io = open("extract/$filename", "w")
open(filename, "r") do io
    for i in 1:13636 # 162544, 104941, 13636
        # txt = join(txt2triple(readline(io)), '\t') # triple_zh, triple_en
        txt = txt2triple(readline(io))
        txt = txt[1] * '\t' * txt[3]
        write(output_io, txt, "\n")
    end
end
close(output_io)

#### 拆分训练集数据
train_data.txt

In [9]:
zh_source = r"<http://zh.dbpedia.org/resource/(.*)>"
zh_property = r"<http://zh.dbpedia.org/property/(.*)>"
en_source = r"<http://dbpedia.org/resource/(.*)>"
en_property = r"<http://dbpedia.org/property/(.*)>"
"""获取三元组信息"""
function get_info(triple)
    s1 = match(zh_source, first(triple))
    if !isnothing(s1)
        s1 = s1.captures[1]
        p1 = match(zh_property, triple[2]).captures[1]
        s2 = match(zh_source, last(triple)).captures[1]
        return "zh:\t$(s1)\t$(p1)\t$(s2)"
    end
    s1 = match(en_source, first(triple)).captures[1]
    p1 = match(en_property, triple[2]).captures[1]
    s2 = match(en_source, last(triple)).captures[1]
    "en:\t$(s1)\t$(p1)\t$(s2)"
end

"""拆分问题和答案"""
function QandA(txt)
    que, ans = split(txt, '\t')
    ans = split(ans, '#')
    len = Int(length(ans)/3)
    "{$(len)} $(que)\n" * join([get_info(ans[3 * i - 2: 3 * i]) for i in 1:len],'\n')
end

QandA

In [10]:
filename = "train_data.txt"
output_io = open("extract/$filename", "w")
dict = Dict{Char,Int}('2'=>0, '3'=>0)
open(filename, "r") do io
    for i in 1:14262
        println(output_io, QandA(readline(io)))
    end
end
close(output_io)

### 数据分析
1. 知识图谱的三元组只包含这几类
   - `<http://zh.dbpedia.org/resource/{中文关键词}>`
   - `<http://zh.dbpedia.org/property/{中/英文连接词}>`
   - `<http://dbpedia.org/resource/{英文关键词}>`
   - `<http://dbpedia.org/property/{英文连接词}>`
   
   对关键词和连接词做训练，提交答案时再将完整链接还原
2. `train_data.txt` 训练集答案为两组的占三分之二(9739/14262)
3. `triple_zh.txt` 的关键词为中文，但连接词大部分为英文。共 1155 种连接词，英文占 1097（中文只有58），英文关系词总数为 102857
3. `triple_en.txt` 共 1637 种关系词
4. `ILLs(zh-en).txt` 与训练集联系更紧密
   - 比如 `triple_zh.txt` 仅有 1/10 左右
5. 在训练集中的混合模式，比如“主干为汉字，夹杂英文关键词”，关键词很大概率会被提取（需确认是否 100% 适用）

In [190]:
# 统计不重复数目
filename = "triple_en.txt"
txts = read(open("extract/$filename", "r"), String)
triples = [Tuple(split(txt, '\t')) for txt in split(txts, '\n')];
pop!(triples) # 去掉末尾空白行
length(unique([triple[2] for triple in triples]))
# length(unique([triple[2] for triple in triples if all(isletter, triple[2])])) # 纯英关键词的数目

In [101]:
# 检查 ILLs(zh-en).txt 与 triple_zh.txt/triple_en.txt 的关系

# en triples
txts = read(open("extract/triple_en.txt", "r"), String)
en_triples = [Tuple(split(txt, '\t')) for txt in split(txts, '\n')]

# zh triples
txts = read(open("extract/triple_zh.txt", "r"), String)
zh_triples = [Tuple(split(txt, '\t')) for txt in split(txts, '\n')]

# zh-en shifts
txts = read(open("extract/ILLs(zh-en).txt", "r"), String)
zh_en_double = [Tuple(split(txt, '\t')) for txt in split(txts, '\n')]
zh_dicts = Set{String}(last.(zh_en_triples))
en_dicts = Set{String}(first.(zh_en_triples))

Set{String} with 13250 elements:
  "Tencent"
  "Altamira_do_Paraná"
  "New_London_County,_Connecticut"
  "Priapus"
  "Xi_County,_Henan"
  "Jataí"
  "Li_Yeguang"
  "Casablanca"
  "Association_of_American_Universities"
  "Badules"
  "Tiszabercel"
  "Lexus"
  "Joseph_McCarthy"
  "Tangier,_Virginia"
  "Luxor"
  "Beire_(Paredes)"
  "Cleveland,_Utah"
  "N-I_(rocket)"
  "Herentals"
  "Filipe_Nyusi"
  "Province_of_Jaén_(Spain)"
  "Roman_Catholic_Diocese_of_Dunkeld"
  "Argente"
  "Sikorsky_(crater)"
  "Chien-Shiung_Wu"
  ⋮ 

In [107]:
count(i -> first(i) in zh_dicts || last(i) in zh_dicts, zh_en_triples)

13637

In [108]:
for (i, (k1, _, k2)) in enumerate(zh_triples)
    k1 in zh_dicts || k2 in zh_dicts || println(join([i, k1, k2], " "))
end

13 袁世凯 宣統帝
15 袁世凯 陸徵祥
17 袁世凯 北洋軍閥
18 袁世凯 共和黨_(中國)
19 袁世凯 儒教
21 勒内·笛卡尔 罗马公教
22 勒内·笛卡尔 阿·卡加里
24 勒内·笛卡尔 亚里士多德
25 勒内·笛卡尔 安瑟伦
26 勒内·笛卡尔 阿奎那
27 勒内·笛卡尔 奥古斯丁
28 勒内·笛卡尔 奥卡姆
29 勒内·笛卡尔 苏瓦雷茨
30 勒内·笛卡尔 馬蘭·梅森
31 勒内·笛卡尔 恩披里克
32 勒内·笛卡尔 蒙田
33 勒内·笛卡尔 鄧斯·司各脫
34 勒内·笛卡尔 斯宾诺莎
35 勒内·笛卡尔 霍布斯
36 勒内·笛卡尔 阿尔诺
37 勒内·笛卡尔 马勒伯
38 勒内·笛卡尔 帕斯卡
39 勒内·笛卡尔 洛克
40 勒内·笛卡尔 莱布尼茨
41 勒内·笛卡尔 喬治·愛德華·摩爾
42 勒内·笛卡尔 康德
43 勒内·笛卡尔 胡塞尔
44 勒内·笛卡尔 萨特
45 勒内·笛卡尔 布伦士维格
46 勒内·笛卡尔 乔姆斯基
47 阿基米德 敘拉古
48 阿基米德 敘拉古
143 圣赫勒拿 詹姆斯敦_(圣赫勒拿)
144 圣赫勒拿 英国女王
145 圣赫勒拿 伊丽莎白二世
146 圣赫勒拿 聖赫勒拿總督
147 圣赫勒拿 麗莎·菲利普斯_(總督)
148 圣赫勒拿 圣赫勒拿镑
163 延安市 杨家岭中央大礼堂
164 延安市 姚引良
165 延安市 薛占海
166 延安市 榆林市
167 延安市 铜川市
192 华中科技大学 丁烈云
193 华中科技大学 中俄工科大学联盟
194 华中科技大学 七校联合办学
195 海因里希·海涅 大学
247 理查德·尼克松 杰拉尔德·福特
250 理查德·尼克松 加利福尼亚州
251 理查德·尼克松 谢里登·唐尼
252 理查德·尼克松 托马斯·库切尔
253 理查德·尼克松 加利福尼亚州第十二国会选区
254 理查德·尼克松 杰里·沃里斯
255 理查德·尼克松 帕特里克·J·希林斯
256 理查德·尼克松 共和党_(美国)
257 理查德·尼克松 特里西娅·尼克松·考克斯
258 理查德·尼克松 朱莉·尼克松·艾森豪威尔
259 理查德·尼克松 惠提尔学院
260 理查德·尼克松 文学士
261 理查德·尼克松 杜克大学法学院
263 理查德·尼克松 贵格会
264 理查德·尼克松 第二次世界大战
26

Excessive output truncated after 524313 bytes.

32310 蒙费梅伊 蒙费梅伊县
32312 貝斯納爾綠地 Bethnal_Green_and_Bow_(UK_Parliament_constituency)
32314 恩里科·莱塔 安傑利諾·阿爾法諾
32315 恩里科·莱塔 乔治·纳波利塔诺
32316 恩里科·莱塔 馬利奧·蒙蒂
32317 恩里科·莱塔 马泰奥·伦齐
32318 恩里科·莱塔 马西莫·达莱马
32319 恩里科·莱塔 朱利亚诺·阿马托
32321 恩里科·莱塔 兰贝托·迪尼
32322 恩里科·莱塔 Patrizia_Toia
32323 恩里科·莱塔 意大利众议院
32324 恩里科·莱塔 基吉宮
32325 恩里科·莱塔 罗马
32326 恩里科·莱塔 比萨大学
32328 恩里科·莱塔 Pisa
32329 恩里科·莱塔 Tuscany


LoadError: BoundsError: attempt to access Tuple{SubString{String}} at index [2]